# Day 8: Pytest & Python Typing - Exercises

This notebook contains exercises for both pytest testing and Pydantic data validation.

## Setup

Install required packages:
```bash
pip install pytest pydantic
```

---

# Part A: Pytest

These exercises focus on writing tests using pytest. Since pytest runs from the terminal (not inside notebooks), you'll create test files in a project folder.

## Project Setup

Create a project folder structure for a **temperature converter**:

```
temp_converter/
├── converter.py       # Main module with conversion functions
└── tests/
    ├── __init__.py
    ├── test_basic.py       # Basic tests and assertions
    ├── test_fixtures.py    # Fixture examples
    └── test_parametrize.py # Parametrization examples
```

### Running Tests

From the `temp_converter/` directory:
```bash
# Run all tests
pytest

# Run with verbose output
pytest -v

# Run with print statements visible
pytest -s

# Run specific test file
pytest tests/test_basic.py

# Run tests matching pattern
pytest -k "celsius"
```

---

## Exercise 1: Create Module and Basic Tests

**Objective:** Create a simple module and write basic tests with proper naming and assertions.

### Part 1: Create the Module

Create `temp_converter/converter.py` with these functions:

1. `celsius_to_fahrenheit(celsius)` - Convert Celsius to Fahrenheit
   - Formula: `(celsius * 9/5) + 32`
   - Returns: float

2. `fahrenheit_to_celsius(fahrenheit)` - Convert Fahrenheit to Celsius
   - Formula: `(fahrenheit - 32) * 5/9`
   - Returns: float

3. `is_freezing(celsius)` - Check if temperature is at or below freezing
   - Returns: True if celsius <= 0, False otherwise

### Part 2: Create Basic Tests

Create `temp_converter/tests/test_basic.py` with:

1. Test file must start with `test_` prefix
2. Import the converter module
3. Write these test functions (all must start with `test_`):
   - `test_celsius_to_fahrenheit_zero()` - Test 0°C = 32°F
   - `test_celsius_to_fahrenheit_hundred()` - Test 100°C = 212°F
   - `test_freezing_point()` - Test `is_freezing(0)` returns True
   - `test_above_freezing()` - Test `is_freezing(10)` returns False

**Hint:** Use simple `assert` statements for equality checks

---

## Exercise 2: Different Types of Assertions

**Objective:** Practice various assertion types: equality, comparison, boolean, membership, and collections.

**Requirements:**

Continue in `temp_converter/tests/test_basic.py` or create new test functions:

### 1. Equality Assertions
- Test `celsius_to_fahrenheit(-40)` equals `-40` (same in both scales)

### 2. Comparison Assertions
- Test `fahrenheit_to_celsius(100)` is greater than 30
- Test `fahrenheit_to_celsius(0)` is less than 0

### 3. Boolean Assertions
- Test `is_freezing(-5)` is `True`
- Test `is_freezing(25)` is `False`

### 4. Membership Assertions
- Create a list: `common_temps = [0, 32, 100, 212]`
- Test that `32` is in the list
- Test that `50` is not in the list

### 5. Collection Assertions
- Test length of `common_temps` equals 4
- Create converted list and verify its type

**Running:**
```bash
pytest tests/test_basic.py -v
```

**Hint:** Use `assert ==`, `assert >`, `assert <`, `assert in`, `assert not in`, `len()`

---

## Exercise 3: Reusable Fixtures

**Objective:** Create reusable fixtures to eliminate duplicate test setup code.

**Requirements:**

Create `temp_converter/tests/test_fixtures.py` demonstrating fixtures:

### Part 1: Show the Problem (Without Fixtures)

First, write tests WITHOUT fixtures to show the repetition problem:

```python
def test_without_fixture_1():
    temps = [0, 25, 100, -10]  # Repeated setup
    # Test conversions using temps

def test_without_fixture_2():
    temps = [0, 25, 100, -10]  # Repeated again!
    # Test different aspect
```

### Part 2: Solve with Fixtures

Create fixtures at the top of the test file:

```python
import pytest

@pytest.fixture
def sample_celsius_temps():
    """Provide sample Celsius temperatures."""
    return [0, 25, 100, -10]

@pytest.fixture
def sample_fahrenheit_temps():
    """Provide sample Fahrenheit temperatures."""
    return [32, 77, 212, 14]
```

Then use them in test functions:

```python
def test_with_fixture_1(sample_celsius_temps):
    # Use fixture - no setup code needed!
    assert len(sample_celsius_temps) == 4

def test_with_fixture_2(sample_celsius_temps):
    # Reuse same fixture
    assert 0 in sample_celsius_temps
```

**Your Task:**
1. Create the test file with both approaches (with and without fixtures)
2. Write at least 3 tests using the fixtures
3. Notice how fixtures eliminate duplicate code

**Hint:** Fixtures are passed as function arguments to tests

---

## Exercise 4: Test Markers - Skip and XFail

**Objective:** Use pytest markers to skip tests conditionally or mark them as expected to fail.

**Requirements:**

Add to `temp_converter/tests/test_basic.py`:

### 1. Skip Tests Unconditionally

Use `@pytest.mark.skip` to skip tests that aren't ready:

```python
import pytest

@pytest.mark.skip(reason="Feature not implemented yet")
def test_kelvin_conversion():
    # This test will be skipped
    result = celsius_to_kelvin(0)
    assert result == 273.15
```

### 2. Skip Tests Conditionally

Use `@pytest.mark.skipif` to skip based on conditions:

```python
import sys

@pytest.mark.skipif(sys.version_info < (3, 10), reason="Requires Python 3.10+")
def test_modern_union_syntax():
    # Only runs on Python 3.10+
    def get_temp(value: int | None) -> int | None:
        return value
    assert get_temp(25) == 25
```

### 3. Expected Failures (XFail)

Use `@pytest.mark.xfail` for tests that are expected to fail:

```python
@pytest.mark.xfail(reason="Known bug in edge case")
def test_extreme_temperature():
    # This test is expected to fail
    result = celsius_to_fahrenheit(1000000)
    assert result < 10000000  # Might fail due to overflow
```

### 4. XFail with Strict Mode

```python
@pytest.mark.xfail(strict=True, reason="Must fix this bug")
def test_negative_absolute_zero():
    # If this passes, test fails (strict mode)
    # If this fails, test passes (expected)
    result = celsius_to_fahrenheit(-300)  # Below absolute zero
    assert result < -459.67
```

### Your Task:

Create tests demonstrating:
1. One unconditional skip
2. One conditional skip (e.g., skip on Windows: `sys.platform == "win32"`)
3. One xfail test
4. One xfail with `strict=True`

### Run and observe:

```bash
pytest tests/test_basic.py -v
```

**Output will show:**
- `SKIPPED` - Test was skipped
- `XFAIL` - Test failed as expected
- `XPASS` - Test passed but was expected to fail (strict mode fails)

**Key Points:**
- `skip`: Temporarily disable tests
- `skipif`: Conditionally skip based on platform, Python version, etc.
- `xfail`: Mark known failures without breaking the test suite
- `strict=True`: Fail if the test unexpectedly passes

**Hint:** Import `pytest` and `sys` at the top of your test file

---

## Exercise 5: Parametrization (Complete Coverage)

**Objective:** Master all parametrization patterns: basic, multiple args, IDs, multiple decorators, and with fixtures.

**Requirements:**

Create `temp_converter/tests/test_parametrize.py` covering all patterns:

### 1. Basic Parametrization
```python
@pytest.mark.parametrize("celsius, expected_fahrenheit", [
    (0, 32),
    (100, 212),
    (-40, -40),
])
def test_basic_conversion(celsius, expected_fahrenheit):
    # Test celsius to fahrenheit conversion
```

### 2. Multiple Arguments
```python
@pytest.mark.parametrize("fahrenheit, expected_celsius, is_cold", [
    (32, 0, True),
    (212, 100, False),
    (14, -10, True),
])
def test_multiple_args(fahrenheit, expected_celsius, is_cold):
    # Test conversion AND freezing check
```

### 3. Using IDs for Readability
```python
@pytest.mark.parametrize(
    "temp",
    [0, 25, 100],
    ids=["freezing_point", "room_temperature", "boiling_point"]
)
def test_with_ids(temp):
    # Test conversions with readable test names
```

### 4. Multiple Decorators (Cartesian Product)
```python
@pytest.mark.parametrize("celsius", [0, 100])
@pytest.mark.parametrize("offset", [0, 5, -5])
def test_multiple_decorators(celsius, offset):
    # Tests all combinations: 2 × 3 = 6 tests total
    adjusted_temp = celsius + offset
    # Test conversion of adjusted temperature
```

### 5. Parametrization with Fixtures
```python
@pytest.mark.parametrize("index, expected_celsius", [
    (0, 0),
    (1, 25),
    (2, 100),
])
def test_with_fixture(sample_fahrenheit_temps, index, expected_celsius):
    # Access fixture AND use parameters
    fahrenheit = sample_fahrenheit_temps[index]
    # Convert and test
```

**Run all parametrized tests:**
```bash
pytest tests/test_parametrize.py -v
```

**Analogy:** Think of parametrization like testing the same device with different inputs - same test logic, different data!

**Hint:** The `-v` flag shows each parametrized test case separately

---

# Part B: Python Typing & Pydantic (5 Exercises)

These exercises focus on type hints and Pydantic for data validation. Complete them directly in this notebook.

---

## Exercise 6: Type Hinting - Basics, Limitations & Dataclasses

**Objective:** Understand the importance of type hints, their limitations, and explore basic types including dataclasses.

### Part 1: Why Type Hints Matter

Create a function WITHOUT type hints and show the problem:

```python
def calculate_total(price, quantity):
    return price * quantity

# What types should price and quantity be? Not clear!
```

**Requirements:**
1. Create the same function WITH type hints
2. Use built-in types: `int`, `float`, `str`, `bool`
3. Add return type annotation

### Part 2: Advanced Types (Optional, List, Dict, Union)

Create functions using:
- `Optional[str]` - value can be string or None
- `list[int]` - list of integers
- `dict[str, float]` - dictionary with string keys and float values
- `str | None` - modern union syntax (Python 3.10+)

**Example signature to implement:**

```python
def process_items(
    items: list[str],
    quantities: dict[str, int],
    notes: str | None = None
) -> list[tuple[str, int]]:
    # Your implementation
```

**Hint:** Import `Optional` from `typing` module, use `list[type]` for lists

### Part 3: Type Hint Limitations

Demonstrate that Python doesn't enforce types at runtime:

```python
def add_numbers(a: int, b: int) -> int:
    return a + b

# This will NOT raise an error!
result = add_numbers("hello", "world")
print(result)  # "helloworld" - wrong type accepted!
```

**Show the problem:** Type hints are just suggestions, not enforced.

### Part 4: Dataclasses (Brief Introduction)

Create a dataclass and show its limitations:

```python
from dataclasses import dataclass

@dataclass
class Product:
    name: str
    price: float
    in_stock: bool
```

**Demonstrate limitations:**
1. Create a Product with wrong types - it accepts them!
2. No automatic validation
3. This is where Pydantic solves the problem

In [ ]:
# Your code here

---

## Exercise 7: Pydantic Basics - BaseModel & Type Validation

**Objective:** Create your first Pydantic model and see how it solves type hint limitations through automatic validation.

### The Problem (from Exercise 6)

Recall that dataclasses don't validate:
```python
@dataclass
class Product:
    name: str
    price: float

# Accepts wrong types!
product = Product(name=123, price="not a number")
```

### The Solution: Pydantic BaseModel

**Requirements:**



1. **Create a Pydantic model** `Product` that inherits from `BaseModel`:- Access error details with `e.errors()`

   - `name` (str)- Use `try/except ValidationError as e`

   - `price` (float)- Import `BaseModel` and `ValidationError` from `pydantic`

   - `in_stock` (bool)**Hint:** 



2. **Create a valid product** with correct types:**Key Point:** Pydantic validates types automatically when creating instances!

   - name: "Laptop"

   - price: 999.99   - Explain why Pydantic is better for data validation

   - in_stock: True   - Show that dataclass ACCEPTS invalid types

   - Show that Pydantic REJECTS invalid types

3. **Demonstrate automatic type validation** - try creating a product with wrong types:4. **Compare with dataclass**:

   - Pass `name` as an integer (123)

   - Pass `price` as a list ([100, 200])   - Show the error details
   - Catch and display the `ValidationError`

In [ ]:
# Your code here

---

## Exercise 8: Type Coercion in Pydantic

**Objective:** Understand Pydantic's automatic type conversion (coercion) for compatible types.

### What is Type Coercion?

Pydantic automatically converts compatible types:
- String "123" → Integer 123
- String "19.99" → Float 19.99
- Integer 1 → Boolean True
- String "yes" → Boolean True (non-empty strings)

**Requirements:**

1. **Create a Pydantic model** `Transaction` with:
   - `transaction_id` (int)
   - `amount` (float)
   - `completed` (bool)

   - `description` (str)

- Compare behavior: some strings convert to int, others don't

2. **Demonstrate successful coercion** - create instances passing wrong types that CAN be coerced:- Use `type(instance.field)` to verify the actual type after coercion

   - Pass `transaction_id` as string: "12345"**Hint:** 

   - Pass `amount` as string: "99.99"

   - Pass `completed` as integer: 1 (or 0 for False)**Key Insight:** Pydantic tries to convert compatible types but rejects incompatible ones.

   - Pass `description` as integer: 789 (if it can be stringified)

      - Confirm "99.99" (str) became 99.99 (float)

   Show that Pydantic accepts and converts them!   - Confirm "123" (str) became 123 (int)

   - After successful coercion, check the actual type using `type()`

3. **Show what CANNOT be coerced** - try creating instances with incompatible types:4. **Verify coerced types**:

   - Pass `transaction_id` as string "abc" (cannot convert to int)

   - Pass `amount` as list [10, 20] (cannot convert to float)   - Catch the ValidationError

In [ ]:
# Your code here

---

## Exercise 9: Parsing/Loading from JSON and Dict

**Objective:** Load and validate data from dictionaries and JSON strings using Pydantic.

### Use Case

Common scenario: API responses, database records, config files all use JSON/dict format.

**Requirements:**

1. **Create a Pydantic model** `User` with:
   - `user_id` (int)
   - `username` (str)
   - `email` (str)
   - `is_active` (bool)
   - `age` (int | None) - optional, can be None

2. **Parse from Dictionary:**
   - Create a dict with valid user data
   - Load it into User model using `**dict` unpacking: `User(**user_dict)`
   - Show that validation happens during parsing
   
3. **Parse from JSON string:**
   - Create a JSON string with user data
   - Parse using `User.model_validate_json(json_string)`

   - Show successful parsing and validation

- ValidationError shows what went wrong during parsing

4. **Demonstrate validation during parsing:**- Use `**dict` unpacking for dictionaries

   - Try parsing a dict with invalid data (wrong types)- Use `model_validate_json()` for JSON strings

   - Try parsing JSON with missing required fields**Hint:**

   - Catch ValidationError for both cases

   - Show error messages**Key Insight:** Pydantic validates data when loading from dict/JSON, ensuring data integrity!



5. **Export back (optional):**   - Show JSON with pretty printing: `user.model_dump_json(indent=2)`

   - Export to dict: `user.model_dump()`   - Export to JSON: `user.model_dump_json()`

In [ ]:
# Your code here

---

## Exercise 10: Additional Validation - EmailStr & Custom Validators

**Objective:** Use Pydantic's built-in EmailStr validator and create custom validation rules.

### Part 1: EmailStr - Built-in Email Validation

**Requirements:**

1. **Create a model** `UserRegistration` with:
   - `username` (str)
   - `email` (EmailStr) - special Pydantic type
   - `age` (int)

2. **Test EmailStr validation:**
   - Create user with valid email: "user@example.com"
   - Try invalid emails and catch ValidationError:
     - "not-an-email" (missing @)
     - "user@" (incomplete domain)
     - "@example.com" (missing username)
   - Show error messages

**Note:** EmailStr requires: `pip install email-validator`

### Part 2: Custom Validators (Optional)

Add custom validation rules using `@field_validator` decorator:

**Requirements:**

1. **Username validator:**
   - Minimum length: 3 characters
   - Maximum length: 20 characters
   - Convert to lowercase automatically
   - No spaces allowed

2. **Age validator:**
   - Minimum: 13 years old
   - Maximum: 120 years old
   - Must be positive integer

**Validator Pattern:**
```python
from pydantic import field_validator

class UserRegistration(BaseModel):
    username: str
    email: EmailStr
    age: int
    
    @field_validator('username')
    @classmethod
    def validate_username(cls, value: str) -> str:
        # Validation logic here
        if len(value) < 3:
            raise ValueError('Username must be at least 3 characters')
        # Can transform value
        return value.lower()
```

**Test your validators:**
- Valid data: username="Alice", email="alice@example.com", age=25
- Invalid username: "ab" (too short)
- Invalid age: 10 (too young)
- Invalid email: "invalid"

**Key Insight:** 
- EmailStr provides automatic email validation
- Custom validators let you add business rules
- Validators can both check AND transform values

**Hint:** 
- Use `@field_validator('field_name')` with `@classmethod`
- Raise `ValueError` with descriptive message for validation failures
- Return the value (transformed or unchanged) from validator

**Hint:** Use `@field_validator('field_name')` decorator with `@classmethod`

In [ ]:
# Your code here